**Run the notebook in conda tf env** <br>
**Conda activate tf**

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sp
import seaborn as sns
# from node2vec import Node2Vec
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score
from pathlib import Path
import pickle
import glob

In [ ]:
from __future__ import division
from __future__ import print_function
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(0)

In [ ]:
from vgae.optimizer import OptimizerAE, OptimizerVAE
from vgae.model import GCNModelVAE, GCNModelAE
from vgae.preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges_seed

In [ ]:
import scipy.sparse as sp
import scipy.stats as scs

## Initialize VGAE Model

In [ ]:
# Calculate ROC AUC
def get_roc_score_vgae(edges_pos, edges_neg, pos_weight, norm, emb=None):
    if emb is None:
        feed_dict.update({placeholders['dropout']: 0})
        emb = sess.run(model.z_mean, feed_dict=feed_dict)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds_pos = []
    pos = []
    for e in edges_pos:
        preds_pos.append(sigmoid(adj_rec[e[0], e[1]])) # predicted score for given edge
        pos.append(adj_orig[e[0], e[1]]) # actual value (1)

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]])) # predicted score for given edge
        neg.append(adj_orig[e[0], e[1]]) # actual value (0)

    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    fpr, tpr, _ = roc_curve(labels_all, preds_all)
    precision, recall, _ = precision_recall_curve(labels_all, preds_all)
    pr_score = auc(recall, precision)
    ap_score = average_precision_score(labels_all, preds_all)
    
    # val_loss = norm * tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels=tf.constant(labels_all), logits=tf.constant(preds_all), pos_weight=pos_weight))
    # val_loss = tf.keras.backend.eval(val_loss)

    return roc_score, ap_score, fpr, tpr, pr_score, precision, recall

In [ ]:
species = ['Homo sapiens','Mus musculus','Rattus norvegicus','Escherichia coli','Bos taurus','Pseudomonas aeruginosa','Arabidopsis thaliana','Saccharomyces cerevisiae','Drosophila melanogaster','Caenorhabditis elegans']
seed_number = [12345, 22345, 32345, 42345, 52345]
index = 0
result = {}
for spcs in species[:1]:
    spcs_temp = {}
    for seed_num in seed_number:
        print(spcs,seed_num)
        # row = [[]]*len(list(result_charc))
        mpi_file_name = './features/mpi_network/'+'mpi_'+str(spcs).replace(' ','_')+'.pkl'
        df_file_name = './features/mpi_features/'+'feature_df_'+str(spcs).replace(' ','_')+'.pkl'
        g = pickle.load(open(mpi_file_name, "rb" ))
        node_feats = pickle.load(open(df_file_name, "rb" ))
        features = np.stack(node_feats['features'].values, axis=0)
        adj = nx.adjacency_matrix(g,nodelist=node_feats['node'].tolist())
        # features = np.random.rand(adj.shape[0],1024)
        adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_seed(adj, test_frac=.1, val_frac=.1,seed=seed_num)
        
        x = sp.lil_matrix(features)
        features_tuple = sparse_to_tuple(x)
        features_shape = features_tuple[2]
        # Get graph attributes (to feed into model)
        num_nodes = adj.shape[0] # number of nodes in adjacency matrix
        num_features = features_shape[1] # number of features (columsn of features matrix)
        features_nonzero = features_tuple[1].shape[0] # number of non-zero entries in features matrix (or length of values list)
        # Store original adjacency matrix (without diagonal entries) for later
        adj_orig = adj
        adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
        adj_orig.eliminate_zeros()
        # Normalize adjacency matrix
        adj_norm = preprocess_graph(adj_train)
        # Add in diagonals
        adj_label = adj_train + sp.eye(adj_train.shape[0])
        adj_label = sparse_to_tuple(adj_label)
        cost_val = []
        acc_val = []
        val_roc_score = []
        train_loss_log = []
        val_loss_log = []
        # Define hyperparameters
        LEARNING_RATE = 0.005
        EPOCHS = 500
        HIDDEN1_DIM = 32
        HIDDEN2_DIM = 16
        DROPOUT = 0.1
        # Define placeholders
        placeholders = {
            'features': tf.sparse_placeholder(tf.float32),
            'adj': tf.sparse_placeholder(tf.float32),
            'adj_orig': tf.sparse_placeholder(tf.float32),
            'dropout': tf.placeholder_with_default(0., shape=())
        }

        # How much to weigh positive examples (true edges) in cost print_function
        # Want to weigh less-frequent classes higher, so as to prevent model output bias
        # pos_weight = (num. negative samples / (num. positive samples)
        pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()

        # normalize (scale) average weighted cost
        norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

        # Create VAE model
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero, HIDDEN1_DIM, HIDDEN2_DIM)

        opt = OptimizerVAE(preds=model.reconstructions,
                                labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices=False), [-1]),
                                model=model, num_nodes=num_nodes,
                                pos_weight=pos_weight,
                                norm=norm,
                                learning_rate=LEARNING_RATE)

        # Initialize session
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        # Train model
        for epoch in range(EPOCHS):
            t = time.time()
            # Construct feed dictionary
            feed_dict = construct_feed_dict(adj_norm, adj_label, features_tuple, placeholders)
            feed_dict.update({placeholders['dropout']: DROPOUT})
            # Run single weight update
            outs = sess.run([opt.opt_op, opt.cost, opt.accuracy], feed_dict=feed_dict)

            # Compute average loss
            avg_cost = outs[1]
            avg_accuracy = outs[2]
            roc_curr, ap_curr, fpr, tpr, pr_score, precision, recall = get_roc_score_vgae(train_edges, train_edges_false, pos_weight, norm)
            # train_loss_log.append(train_loss_epoch)

            # # Evaluate predictions
            roc_curr, ap_curr, fpr, tpr, pr_score, precision, recall = get_roc_score_vgae(val_edges, val_edges_false, pos_weight, norm)
            val_roc_score.append(roc_curr)
            # val_loss_log.append(val_loss_epoch)
            
            if (epoch+1)%50 == 0:
            # Print results for this epoch
                print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                    "train_acc=", "{:.5f}".format(avg_accuracy), "val_roc=", "{:.5f}".format(val_roc_score[-1]),
                    "val_ap=", "{:.5f}".format(ap_curr),
                    "time=", "{:.5f}".format(time.time() - t))

        print("Optimization Finished!")
        # Print final results
        feature_roc_score, feature_ap_score, feature_fpr, feature_tpr, feature_pr_score, feature_prcs, feature_rcal = get_roc_score_vgae(test_edges, test_edges_false, pos_weight, norm)
        temp = [feature_roc_score, feature_ap_score, feature_fpr, feature_tpr, feature_pr_score, feature_prcs, feature_rcal]
        print('Test ROC score: ' + str(feature_roc_score))
        print('Test AP score: ' + str(feature_ap_score))
        spcs_temp[seed_num] = temp
    result[spcs] = spcs_temp
pickle.dump(result,open('./graph_model_vgae_features.pkl','wb'))